# relation合并到MEU转xlsx

In [9]:
import os
import pandas as pd
from collections import defaultdict

def process_files_xlsx_input( # Renamed function for clarity
        relation_dir=r'law_to_ComplianceUnit/st_3_ComplianceUnit_relations',
        cu_dir=r'law_to_ComplianceUnit/st_2_ComplianceUnit',
        output_dir=r'law_to_ComplianceUnit/st_3_ComplianceUnit_relations/ComplianceUnit_with_relation'
):
    print(f"程序开始处理...")
    print(f"关系文件目录 (CSV): {relation_dir}")
    print(f"CU 文件目录 (XLSX): {cu_dir}") # Clarified input type
    print(f"输出文件目录: {output_dir}")

    # 1. 检查输入目录是否存在
    if not os.path.isdir(relation_dir):
        print(f"错误：关系文件目录 '{relation_dir}' 不存在或不是一个目录。程序将退出。")
        return
    if not os.path.isdir(cu_dir):
        print(f"错误：CU 文件目录 '{cu_dir}' 不存在或不是一个目录。程序将退出。")
        return

    # 2. 确保输出目录存在
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"输出目录 '{output_dir}' 已确保存在。")
    except OSError as e:
        print(f"错误：创建输出目录 '{output_dir}' 失败: {e}。程序将退出。")
        return

    processed_files_count = 0
    skipped_files_count = 0

    # 遍历 cu_dir 目录中的所有 .xlsx 文件
    try:
        # 修改此处以查找 .xlsx 文件
        cu_filenames = [f for f in os.listdir(cu_dir) if f.endswith(".xlsx")]
    except FileNotFoundError:
        print(f"错误: 无法访问 CU 目录 '{cu_dir}'。可能路径不正确或权限不足。程序将退出。")
        return
    except Exception as e:
        print(f"错误: 遍历 CU 目录 '{cu_dir}' 时发生意外错误: {e}。程序将退出。")
        return

    if not cu_filenames:
        print(f"警告: 在 CU 目录 '{cu_dir}' 中没有找到 .xlsx 文件。程序将结束。") # Updated message
        return

    print(f"\n开始处理 '{cu_dir}' 中的 {len(cu_filenames)} 个 XLSX 文件...\n") # Updated message

    for filename in cu_filenames:
        print(f"--- 开始处理文件: {filename} ---")
        error_occurred_for_file = False
        relation_df = pd.DataFrame()
        cu_df = pd.DataFrame()

        # CU 文件名现在是 .xlsx, 关系文件名应保持一致 (不含扩展名部分)
        base_filename_no_ext = os.path.splitext(filename)[0]
        relation_filename_csv = base_filename_no_ext + ".csv" # Assume relation file is CSV with same base name

        relation_path = os.path.join(relation_dir, relation_filename_csv)
        cu_path = os.path.join(cu_dir, filename) # filename already includes .xlsx
        output_filename_base = base_filename_no_ext # For output .xlsx

        # 3. 检查各个文件是否存在
        if not os.path.exists(cu_path):
            print(f"警告：文件 '{filename}' 在 CU 目录中列出，但实际未找到于 '{cu_path}'。跳过此文件。")
            skipped_files_count += 1
            print(f"--- 文件 {filename} 处理失败 ---\n")
            continue

        if not os.path.exists(relation_path):
            print(f"警告：未找到CU文件 '{filename}' 对应的关系文件 '{relation_path}'。跳过此文件。")
            skipped_files_count += 1
            print(f"--- 文件 {filename} 处理失败 ---\n")
            continue

        # 4. 读取关系数据 (CSV)
        try:
            relation_df = pd.read_csv(relation_path, encoding="utf-8-sig")
            if relation_df.empty:
                print(f"信息：关系文件 '{relation_path}' 为空。将继续处理，但可能没有关系数据。")
        except pd.errors.EmptyDataError:
            print(f"信息：关系文件 '{relation_path}' 为空 (EmptyDataError)。将继续处理。")
            relation_df = pd.DataFrame()
        except pd.errors.ParserError:
            print(f"错误：解析关系文件 '{relation_path}' 失败 (ParserError)，请检查文件格式是否为有效的CSV。跳过此文件。")
            error_occurred_for_file = True
        except UnicodeDecodeError:
            print(f"错误：解码关系文件 '{relation_path}' 失败 (UnicodeDecodeError)。请确保文件编码为 utf-8-sig。跳过此文件。")
            error_occurred_for_file = True
        except Exception as e:
            print(f"错误：读取关系文件 '{relation_path}' 时发生未知错误: {e}。跳过此文件。")
            error_occurred_for_file = True

        if error_occurred_for_file:
            skipped_files_count += 1
            print(f"--- 文件 {filename} 处理失败 ---\n")
            continue

        # 读取CU数据 (XLSX)
        try:
            # 修改此处以读取 .xlsx 文件
            cu_df = pd.read_excel(cu_path, engine='openpyxl') # Specify engine for robustness
            if cu_df.empty:
                print(f"信息：CU 文件 (Excel) '{cu_path}' 为空或第一个工作表为空。")
        # except pd.errors.EmptyDataError: # read_excel usually returns empty DataFrame, not this error
        #     print(f"信息：CU 文件 (Excel) '{cu_path}' 为空 (EmptyDataError)。")
        #     cu_df = pd.DataFrame()
        except FileNotFoundError: # Should be caught by os.path.exists, but as safeguard
            print(f"错误：读取CU文件 '{cu_path}' 时发生 FileNotFoundError。跳过此文件。")
            error_occurred_for_file = True
        except Exception as e: # Catch other potential errors like bad Excel format, openpyxl issues
            print(f"错误：读取或解析CU Excel文件 '{cu_path}' 时发生错误: {e}。请确保文件是有效的Excel格式且 'openpyxl' 已安装。跳过此文件。")
            error_occurred_for_file = True


        if error_occurred_for_file:
            skipped_files_count += 1
            print(f"--- 文件 {filename} 处理失败 ---\n")
            continue

        # 5. 检查必要的列是否存在
        if not relation_df.empty:
            required_relation_cols = ["source", "relation", "target"]
            missing_relation_cols = [col for col in required_relation_cols if col not in relation_df.columns]
            if missing_relation_cols:
                print(f"错误：关系文件 '{relation_path}' 缺少以下必要列: {', '.join(missing_relation_cols)}。跳过此文件。")
                skipped_files_count += 1
                print(f"--- 文件 {filename} 处理失败 ---\n")
                continue
        
        if not cu_df.empty:
            required_cu_cols = ["cu_id"]
            missing_cu_cols = [col for col in required_cu_cols if col not in cu_df.columns]
            if missing_cu_cols:
                print(f"错误：CU文件 '{cu_path}' 缺少以下必要列: {', '.join(missing_cu_cols)}。跳过此文件。")
                skipped_files_count += 1
                print(f"--- 文件 {filename} 处理失败 ---\n")
                continue

        # 创建关系字典
        relation_dict = defaultdict(lambda: defaultdict(list))
        if not relation_df.empty:
            try:
                for _, row in relation_df.iterrows():
                    source = row["source"]
                    relation_val = row["relation"]
                    target = row["target"]
                    relation_dict[source][relation_val].append(str(target))
            except KeyError as e:
                print(f"错误：处理关系文件 '{relation_path}' 时访问列 '{e}' 失败。跳过此文件。")
                error_occurred_for_file = True # Mark error
            except Exception as e:
                print(f"错误：在 '{relation_path}' 中创建关系字典时发生意外错误: {e}。跳过此文件。")
                error_occurred_for_file = True # Mark error
        
        if error_occurred_for_file: # Check if error occurred during relation_dict creation
            skipped_files_count += 1
            print(f"--- 文件 {filename} 处理失败 ---\n")
            continue

        # 合并数据
        merged_rows = []
        if not cu_df.empty:
            try:
                for _, cu_row in cu_df.iterrows():
                    cu_id = cu_row["cu_id"]
                    base_info = cu_row.to_dict()

                    if cu_id in relation_dict:
                        for rel_type, targets in relation_dict[cu_id].items():
                            new_row = base_info.copy()
                            new_row["relation"] = rel_type
                            new_row["target"] = ";".join(map(str, targets))
                            merged_rows.append(new_row)
                    else:
                        new_row = base_info.copy()
                        new_row["relation"] = ""
                        new_row["target"] = ""
                        merged_rows.append(new_row)
            except KeyError as e:
                print(f"错误：处理CU文件 '{cu_path}' 时访问列 '{e}' (可能是 'cu_id') 失败。跳过此文件。")
                error_occurred_for_file = True
            except Exception as e:
                print(f"错误：在 '{cu_path}' 中合并数据时发生意外错误: {e}。跳过此文件。")
                error_occurred_for_file = True

        if error_occurred_for_file: # Check if error occurred during merge
            skipped_files_count += 1
            print(f"--- 文件 {filename} 处理失败 ---\n")
            continue
        
        if not merged_rows:
            if not error_occurred_for_file:
                print(f"信息：文件 '{filename}' 的CU数据为空或未产生任何可合并的行。将不会生成输出文件。")
                processed_files_count +=1
            print(f"--- 文件 {filename} 处理完毕 (无输出) ---\n")
            continue

        result_df = pd.DataFrame(merged_rows)
        result_df['confirmed'] = 0
        result_df['comments_relation'] = ''
        
        defined_columns = ["cu_id", "subject", "condition", "constraint", "contextual_info", "relation", "target", 'confirmed', 'comments_relation']
        
        final_columns = []
        for col in defined_columns:
            if col in result_df.columns:
                final_columns.append(col)
            else:
                print(f"警告：在为 '{output_filename_base}.xlsx' 生成输出时，预定义列 '{col}' 在结果中缺失，将被省略。")
        
        if "cu_id" not in result_df.columns:
            print(f"错误: 'cu_id' 列在最终的 DataFrame 中缺失，无法排序和保存文件 '{output_filename_base}.xlsx'。跳过此文件。")
            skipped_files_count +=1
            print(f"--- 文件 {filename} 处理失败 ---\n")
            continue
            
        def sort_key_func(cu_id_val):
            try:
                if not isinstance(cu_id_val, str):
                    cu_id_val = str(cu_id_val)
                parts = cu_id_val.split('_')
                if len(parts) >= 3:
                    n = int(parts[1])
                    k = int(parts[2])
                    return (n, k)
                else:
                    return (float('inf'), float('inf'))
            except (IndexError, ValueError):
                return (float('inf'), float('inf'))
            except Exception:
                return (float('inf'), float('inf'))

        try:
            sort_keys_series = result_df["cu_id"].map(sort_key_func)
            result_df = result_df.loc[sort_keys_series.sort_values().index]
        except KeyError:
             print(f"警告: 'cu_id' 列不存在于 '{output_filename_base}.xlsx' 的结果中，无法排序。将按当前顺序保存。")
        except Exception as e:
            print(f"警告：对文件 '{output_filename_base}.xlsx' 的数据进行排序时发生错误: {e}。将尝试按未排序顺序保存。")
        
        output_path_excel = os.path.join(output_dir, output_filename_base + '.xlsx')
        try:
            result_df.to_excel(
                output_path_excel,
                columns=final_columns,
                index=False,
                engine='openpyxl'
            )
            print(f"成功：已处理并保存文件到 '{output_path_excel}'")
            processed_files_count += 1
        except ImportError: # Specifically for openpyxl, though read_excel would likely fail first
            print(f"错误：缺少 'openpyxl' 库，无法保存为 .xlsx 文件。请运行 'pip install openpyxl'。跳过保存 '{output_path_excel}'。")
            if not error_occurred_for_file :
                 skipped_files_count +=1
        except Exception as e:
            print(f"错误：保存 Excel 文件 '{output_path_excel}' 失败: {e}。")
            if not error_occurred_for_file:
                 skipped_files_count += 1

        print(f"--- 文件 {filename} 处理完毕 ---\n")

    print(f"\n--- 处理总结 ---")
    print(f"总共尝试处理 {len(cu_filenames)} 个在 '{cu_dir}' 中找到的XLSX文件。")
    print(f"成功生成并保存输出的文件数: {processed_files_count}")
    print(f"跳过或处理/保存失败的文件数: {skipped_files_count}")
    print(f"程序处理结束。")

# 如果你需要运行这个脚本，可以这样调用：
if __name__ == '__main__':
    
    process_files_xlsx_input() # 使用函数定义的默认路径
    

程序开始处理...
关系文件目录 (CSV): law_to_ComplianceUnit/st_3_ComplianceUnit_relations
CU 文件目录 (XLSX): law_to_ComplianceUnit/st_2_ComplianceUnit
输出文件目录: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/ComplianceUnit_with_relation
输出目录 'law_to_ComplianceUnit/st_3_ComplianceUnit_relations/ComplianceUnit_with_relation' 已确保存在。

开始处理 'law_to_ComplianceUnit/st_2_ComplianceUnit' 中的 10 个 XLSX 文件...

--- 开始处理文件: 北京证券交易所上市公司持续监管指引第5号——要约收购.xlsx ---
成功：已处理并保存文件到 'law_to_ComplianceUnit/st_3_ComplianceUnit_relations/ComplianceUnit_with_relation/北京证券交易所上市公司持续监管指引第5号——要约收购.xlsx'
--- 文件 北京证券交易所上市公司持续监管指引第5号——要约收购.xlsx 处理完毕 ---

--- 开始处理文件: 北京证券交易所上市公司持续监管指引第6号——内幕信息知情人管理及报送.xlsx ---
成功：已处理并保存文件到 'law_to_ComplianceUnit/st_3_ComplianceUnit_relations/ComplianceUnit_with_relation/北京证券交易所上市公司持续监管指引第6号——内幕信息知情人管理及报送.xlsx'
--- 文件 北京证券交易所上市公司持续监管指引第6号——内幕信息知情人管理及报送.xlsx 处理完毕 ---

--- 开始处理文件: 北京证券交易所上市公司持续监管指引第9号——募集资金管理.xlsx ---
成功：已处理并保存文件到 'law_to_ComplianceUnit/st_3_ComplianceUnit_relations/ComplianceUnit_with_re

In [6]:
# 执行处理
process_files()

# 地址结构

In [7]:
import os

def print_directory_tree(
    start_path=None, 
    indent="", 
    show_files=True, 
    prefix=""
):
    """
    打印目录结构树，可选择是否包含文件
    
    Args:
        start_path (str): 起始路径，默认为当前工作目录
        indent (str): 缩进字符（内部递归使用）
        show_files (bool): 是否显示文件，默认 True
        prefix (str): 每行前缀（内部递归使用）
    """
    if start_path is None:
        start_path = os.getcwd()
    
    # 打印当前目录名
    dir_name = os.path.basename(start_path)
    print(prefix + indent + dir_name + "/")
    
    new_indent = indent + "    "
    try:
        items = sorted(os.listdir(start_path))
    except PermissionError:
        print(prefix + new_indent + "[Permission Denied]")
        return
    
    for item in items:
        item_path = os.path.join(start_path, item)
        if os.path.isdir(item_path):
            # 递归处理子目录
            print_directory_tree(
                item_path, 
                new_indent, 
                show_files, 
                prefix
            )
        elif show_files:
            # 如果是文件，并且 show_files=True，则打印
            print(prefix + new_indent + item)


print_directory_tree(
    start_path=None, 
    indent="", 
    show_files=False, 
    # show_files=True, 
    prefix=""
)

FindCheck-pipeline/
    ComplianceUnit_to_code/
        ComplianceUnit_code/
            GT/
            raw_response/
            view_ipynb/
        ComplianceUnit_selected_with_relation_GT/
    data_simulation/
        data_generated/
    law_to_ComplianceUnit/
        st_0_law_docx/
            with_appendix/
        st_1_law_csv/
        st_2_ComplianceUnit/
            GT/
            raw_response/
        st_3_ComplianceUnit_relations/
            ComplianceUnit_with_relation/
                GT/
            raw_response/
        st_5_ComplianceUnit_Graph_HTML/
            GT/
    utils/
        __pycache__/
